In [ ]:
#!python -m pip install ml_metrics

# datasets:
<ul>
<li>
<a href="https://cseweb.ucsd.edu/~jmcauley/datasets.html">page with some datasets</a>
</li>
<li>
<a href="https://github.com/caserec/Datasets-for-Recommender-Systems/blob/master/README.md
">github</a>
</li>
<li>
<a href="http://snap.stanford.edu/data/web-Amazon-links.html">Amazon</a>
</li>
</ul>

In [ ]:
import pandas as pd
import numpy as np

# <a href="https://www.kaggle.com/rounakbanik/the-movies-dataset">The Movies Dataset: </a> 

## Read Data:

In [ ]:
ratings_df=pd.read_csv('movies_45000/ratings_small.csv')
ratings_df.head(3)

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182


In [ ]:
links_df=pd.read_csv('movies_45000/links_small.csv')
links_df.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [ ]:
metadata_df=pd.read_csv('movies_45000/movies_metadata.csv')
metadata_df.head(3)

C:\Users\EQUIPO\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0


## We drop the not used columns, Rename and convert numeric data

In [ ]:
ratings_df = ratings_df.drop(columns='timestamp') #drop timestamp
ratings_df.head(3)

,userId,movieId,rating
0,1,31,2.5
1,1,1029,3.0
2,1,1061,3.0


In [ ]:
metadata_df = metadata_df[['adult','genres', 'id', 'original_language','poster_path','title','overview']] #drop not used columns
metadata_df = metadata_df.rename(columns={'id': 'tmdbId'}) #rename id to merge
metadata_df.head(3)

,adult,genres,tmdbId,original_language,poster_path,title,overview
0,False,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,en,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,Toy Story,"Led by Woody, Andy's toys live happily in his ..."
1,False,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,en,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,Jumanji,When siblings Judy and Peter discover an encha...
2,False,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602,en,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,Grumpier Old Men,A family wedding reignites the ancient feud be...


In [ ]:
metadata_df['tmdbId'] = pd.to_numeric(metadata_df['tmdbId'], errors='coerce') #id to numeric

In [ ]:
metadata_df.head(3)

,adult,genres,tmdbId,original_language,poster_path,title,overview
0,False,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862.0,en,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,Toy Story,"Led by Woody, Andy's toys live happily in his ..."
1,False,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844.0,en,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,Jumanji,When siblings Judy and Peter discover an encha...
2,False,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602.0,en,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,Grumpier Old Men,A family wedding reignites the ancient feud be...


In [ ]:
links_df = links_df.drop(columns='imdbId') #drop not used column

# Merge the colums

In [ ]:
ratings_df = ratings_df.merge(links_df) #merge rating_df with links_df
ratings_df.shape
ratings_df.sort_values(by=['movieId'],ascending=True).head(10) #sort by movie id, just to confirm data integrity

,userId,movieId,rating,tmdbId
31535,586,1,4.0,862.0
31471,434,1,4.0,862.0
31472,437,1,4.0,862.0
31473,440,1,4.0,862.0
31474,442,1,4.0,862.0
31475,447,1,3.0,862.0
31476,448,1,3.0,862.0
31477,452,1,3.5,862.0
31478,455,1,3.5,862.0
31479,456,1,3.5,862.0


In [ ]:
ratings_df = ratings_df.merge(metadata_df) #merge the new ratings_df with metadata_df
#ratings_df.shape
ratings_df = ratings_df.drop(columns='movieId') # movieId is not uset anymore
ratings_df.sort_values(by=['userId'],ascending=True).head(10) #sort by userId to verify data integrity

,userId,rating,tmdbId,adult,genres,original_language,poster_path,title,overview
0,1,2.5,9909.0,False,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...",en,/y5Jee3QmYOlpqfaPPbfvtdVc5wj.jpg,Dangerous Minds,Former Marine Louanne Johnson lands a gig teac...
117,1,2.0,1103.0,False,"[{'id': 878, 'name': 'Science Fiction'}, {'id'...",en,/z1KTbKJh7vSTTXllxxE4DWG63rT.jpg,Escape from New York,"In 1997, the island of Manhattan has been wall..."
165,1,4.0,11216.0,False,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",it,/xKAweeF2ZPMNn6ce4GclSbr59Pv.jpg,Cinema Paradiso,"A filmmaker recalls his childhood, when he fel..."
403,1,2.0,1598.0,False,"[{'id': 80, 'name': 'Crime'}, {'id': 53, 'name...",en,/4KvrvcqckdupXXO2YnANtyG7QLK.jpg,Cape Fear,Sam Bowden is a small-town corporate attorney....
211,1,2.0,11778.0,False,"[{'id': 18, 'name': 'Drama'}, {'id': 10752, 'n...",en,/slNJESItHPqp1CENEJQUPw8d7WE.jpg,The Deer Hunter,A group of working-class friends decides to en...
259,1,2.0,665.0,False,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",en,/syPMBvvZsADTTRu3UKuxO1Wflq.jpg,Ben-Hur,Ben-Hur is a 1959 epic film directed by Willia...
305,1,2.0,783.0,False,"[{'id': 18, 'name': 'Drama'}, {'id': 36, 'name...",en,/2z9A4FSu1YySrhhcuqkdMIXpgyN.jpg,Gandhi,"In the early years of the 20th century, Mohand..."
849,1,3.0,11072.0,False,"[{'id': 37, 'name': 'Western'}, {'id': 35, 'na...",en,/rT8uRG8CVONiQdgdFRafVtjSdb4.jpg,Blazing Saddles,A town – where everyone seems to be named John...
84,1,3.0,819.0,False,"[{'id': 80, 'name': 'Crime'}, {'id': 18, 'name...",en,/cDqEv4Fw4JZh2zCfecqw3z09L8z.jpg,Sleepers,Two gangsters seek revenge on the state jail w...
806,1,1.0,36819.0,False,"[{'id': 10751, 'name': 'Family'}, {'id': 14, '...",en,/4VZtpwdhHQSa4LUkvujyGAHb1hG.jpg,Time Bandits,Young history buff Kevin can scarcely believe ...


# Take the Genres

In [ ]:
metadata_df['genres'] = metadata_df['genres'].str.replace(r'[^\w\s]+', '') #remove punctuation
metadata_df['overview'] = metadata_df['overview'].str.replace(r'[^\w\s]+', '') #remove punctuation
metadata_df

ratings_df['genres'] = ratings_df['genres'].str.replace(r'[^\w\s]+', '') #remove punctuation
ratings_df['overview'] = ratings_df['overview'].str.replace(r'[^\w\s]+', '') #remove punctuation
ratings_df

,userId,rating,tmdbId,adult,genres,original_language,poster_path,title,overview
0,1,2.5,9909.0,False,id 18 name Drama id 80 name Crime,en,/y5Jee3QmYOlpqfaPPbfvtdVc5wj.jpg,Dangerous Minds,Former Marine Louanne Johnson lands a gig teac...
1,7,3.0,9909.0,False,id 18 name Drama id 80 name Crime,en,/y5Jee3QmYOlpqfaPPbfvtdVc5wj.jpg,Dangerous Minds,Former Marine Louanne Johnson lands a gig teac...
2,31,4.0,9909.0,False,id 18 name Drama id 80 name Crime,en,/y5Jee3QmYOlpqfaPPbfvtdVc5wj.jpg,Dangerous Minds,Former Marine Louanne Johnson lands a gig teac...
3,32,4.0,9909.0,False,id 18 name Drama id 80 name Crime,en,/y5Jee3QmYOlpqfaPPbfvtdVc5wj.jpg,Dangerous Minds,Former Marine Louanne Johnson lands a gig teac...
4,36,3.0,9909.0,False,id 18 name Drama id 80 name Crime,en,/y5Jee3QmYOlpqfaPPbfvtdVc5wj.jpg,Dangerous Minds,Former Marine Louanne Johnson lands a gig teac...
...,...,...,...,...,...,...,...,...,...
100058,664,2.5,34812.0,False,id 878 name Science Fiction,en,/l8VVQgtuZIigZCaOLv40zZR1aMT.jpg,H.G. Wells' War of the Worlds,In this modern retelling of HG Wells classic s...
100059,664,3.5,22825.0,False,id 53 name Thriller id 878 name Science Fiction,en,/1zinhNzY0okQVD1aZBZyrLffugu.jpg,The Box,Norma and Arthur Lewis a suburban couple with ...
100060,665,3.0,110972.0,False,id 35 name Comedy id 10749 name Romance,en,/6KO5jsPOr80J8PkKXPE8fym5I2R.jpg,Pie in the Sky,Pie in the Sky is a 1996 American romantic com...
100061,665,1.0,26602.0,False,id 18 name Drama id 35 name Comedy id 10749 na...,en,/ik6vcNPcVb93uuMZHYXLnmOHVB5.jpg,Summer Catch,A comingofage romantic comedy set against the ...


In [ ]:
metadata_df['genres'] = metadata_df['genres'].str.replace('\d+', '') #take just the genres (not id's)
metadata_df

ratings_df['genres'] = ratings_df['genres'].str.replace('\d+', '') #take just the genres (not id's)
ratings_df

,userId,rating,tmdbId,adult,genres,original_language,poster_path,title,overview
0,1,2.5,9909.0,False,id name Drama id name Crime,en,/y5Jee3QmYOlpqfaPPbfvtdVc5wj.jpg,Dangerous Minds,Former Marine Louanne Johnson lands a gig teac...
1,7,3.0,9909.0,False,id name Drama id name Crime,en,/y5Jee3QmYOlpqfaPPbfvtdVc5wj.jpg,Dangerous Minds,Former Marine Louanne Johnson lands a gig teac...
2,31,4.0,9909.0,False,id name Drama id name Crime,en,/y5Jee3QmYOlpqfaPPbfvtdVc5wj.jpg,Dangerous Minds,Former Marine Louanne Johnson lands a gig teac...
3,32,4.0,9909.0,False,id name Drama id name Crime,en,/y5Jee3QmYOlpqfaPPbfvtdVc5wj.jpg,Dangerous Minds,Former Marine Louanne Johnson lands a gig teac...
4,36,3.0,9909.0,False,id name Drama id name Crime,en,/y5Jee3QmYOlpqfaPPbfvtdVc5wj.jpg,Dangerous Minds,Former Marine Louanne Johnson lands a gig teac...
...,...,...,...,...,...,...,...,...,...
100058,664,2.5,34812.0,False,id name Science Fiction,en,/l8VVQgtuZIigZCaOLv40zZR1aMT.jpg,H.G. Wells' War of the Worlds,In this modern retelling of HG Wells classic s...
100059,664,3.5,22825.0,False,id name Thriller id name Science Fiction,en,/1zinhNzY0okQVD1aZBZyrLffugu.jpg,The Box,Norma and Arthur Lewis a suburban couple with ...
100060,665,3.0,110972.0,False,id name Comedy id name Romance,en,/6KO5jsPOr80J8PkKXPE8fym5I2R.jpg,Pie in the Sky,Pie in the Sky is a 1996 American romantic com...
100061,665,1.0,26602.0,False,id name Drama id name Comedy id name Romance,en,/ik6vcNPcVb93uuMZHYXLnmOHVB5.jpg,Summer Catch,A comingofage romantic comedy set against the ...


In [ ]:
metadata_df['genres'] = metadata_df['genres'].str.replace('id', '') #remove id
metadata_df['genres'] = metadata_df['genres'].str.replace('name', '') #remove name  
metadata_df['genres'] = metadata_df['genres'].str.replace('Science Fiction', 'ScienceFiction') #join Science Fiction
metadata_df['genres'] = metadata_df['genres'].str.strip()
metadata_df['genres'] = metadata_df['genres'].str.replace('    ', ' ')
metadata_df

ratings_df['genres'] = ratings_df['genres'].str.replace('id', '') #remove id
ratings_df['genres'] = ratings_df['genres'].str.replace('name', '') #remove name
ratings_df['genres'] = ratings_df['genres'].str.replace('Science Fiction', 'ScienceFiction') #join Science Fiction
ratings_df['genres'] = ratings_df['genres'].str.strip()
ratings_df['genres'] = ratings_df['genres'].str.replace('    ', ' ')
ratings_df

,userId,rating,tmdbId,adult,genres,original_language,poster_path,title,overview
0,1,2.5,9909.0,False,Drama Crime,en,/y5Jee3QmYOlpqfaPPbfvtdVc5wj.jpg,Dangerous Minds,Former Marine Louanne Johnson lands a gig teac...
1,7,3.0,9909.0,False,Drama Crime,en,/y5Jee3QmYOlpqfaPPbfvtdVc5wj.jpg,Dangerous Minds,Former Marine Louanne Johnson lands a gig teac...
2,31,4.0,9909.0,False,Drama Crime,en,/y5Jee3QmYOlpqfaPPbfvtdVc5wj.jpg,Dangerous Minds,Former Marine Louanne Johnson lands a gig teac...
3,32,4.0,9909.0,False,Drama Crime,en,/y5Jee3QmYOlpqfaPPbfvtdVc5wj.jpg,Dangerous Minds,Former Marine Louanne Johnson lands a gig teac...
4,36,3.0,9909.0,False,Drama Crime,en,/y5Jee3QmYOlpqfaPPbfvtdVc5wj.jpg,Dangerous Minds,Former Marine Louanne Johnson lands a gig teac...
...,...,...,...,...,...,...,...,...,...
100058,664,2.5,34812.0,False,ScienceFiction,en,/l8VVQgtuZIigZCaOLv40zZR1aMT.jpg,H.G. Wells' War of the Worlds,In this modern retelling of HG Wells classic s...
100059,664,3.5,22825.0,False,Thriller ScienceFiction,en,/1zinhNzY0okQVD1aZBZyrLffugu.jpg,The Box,Norma and Arthur Lewis a suburban couple with ...
100060,665,3.0,110972.0,False,Comedy Romance,en,/6KO5jsPOr80J8PkKXPE8fym5I2R.jpg,Pie in the Sky,Pie in the Sky is a 1996 American romantic com...
100061,665,1.0,26602.0,False,Drama Comedy Romance,en,/ik6vcNPcVb93uuMZHYXLnmOHVB5.jpg,Summer Catch,A comingofage romantic comedy set against the ...


# One Hot encodign of genres

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
corpus = ratings_df['genres'] #We define the colum to Encode 

vectorizer = CountVectorizer() #We'll use a count vectorizer instance
generes_onehot = vectorizer.fit_transform(corpus) #pass the genres to the instance
tags = vectorizer.get_feature_names() #get the tags (genres)
generes = generes_onehot.toarray() #get the array

In [ ]:
generes_onehot = pd.DataFrame(generes, columns=tags) #create the genres one hot dataframe
generes_onehot 

,action,adventure,animation,aniplex,brosta,carousel,cartel,comedy,committee,crime,...,sentai,state,telescene,the,thriller,tv,view,vision,war,western
0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100058,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
100059,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
100060,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
100061,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
ratings_df = ratings_df.join(generes_onehot) #join the one hot genres to the ratings dataframe
ratings_df.sort_values(by=['userId'],ascending=True).head(10) #sort by user

,userId,rating,tmdbId,adult,genres,original_language,poster_path,title,overview,action,...,sentai,state,telescene,the,thriller,tv,view,vision,war,western
0,1,2.5,9909.0,False,Drama Crime,en,/y5Jee3QmYOlpqfaPPbfvtdVc5wj.jpg,Dangerous Minds,Former Marine Louanne Johnson lands a gig teac...,0,...,0,0,0,0,0,0,0,0,0,0
117,1,2.0,1103.0,False,ScienceFiction Action,en,/z1KTbKJh7vSTTXllxxE4DWG63rT.jpg,Escape from New York,In 1997 the island of Manhattan has been walle...,1,...,0,0,0,0,0,0,0,0,0,0
165,1,4.0,11216.0,False,Drama Romance,it,/xKAweeF2ZPMNn6ce4GclSbr59Pv.jpg,Cinema Paradiso,A filmmaker recalls his childhood when he fell...,0,...,0,0,0,0,0,0,0,0,0,0
403,1,2.0,1598.0,False,Crime Thriller,en,/4KvrvcqckdupXXO2YnANtyG7QLK.jpg,Cape Fear,Sam Bowden is a smalltown corporate attorney M...,0,...,0,0,0,0,1,0,0,0,0,0
211,1,2.0,11778.0,False,Drama War,en,/slNJESItHPqp1CENEJQUPw8d7WE.jpg,The Deer Hunter,A group of workingclass friends decides to enl...,0,...,0,0,0,0,0,0,0,0,1,0
259,1,2.0,665.0,False,Action Adventure Drama History,en,/syPMBvvZsADTTRu3UKuxO1Wflq.jpg,Ben-Hur,BenHur is a 1959 epic film directed by William...,1,...,0,0,0,0,0,0,0,0,0,0
305,1,2.0,783.0,False,Drama History,en,/2z9A4FSu1YySrhhcuqkdMIXpgyN.jpg,Gandhi,In the early years of the 20th century Mohanda...,0,...,0,0,0,0,0,0,0,0,0,0
849,1,3.0,11072.0,False,Western Comedy,en,/rT8uRG8CVONiQdgdFRafVtjSdb4.jpg,Blazing Saddles,A town where everyone seems to be named Johns...,0,...,0,0,0,0,0,0,0,0,0,1
84,1,3.0,819.0,False,Crime Drama Thriller,en,/cDqEv4Fw4JZh2zCfecqw3z09L8z.jpg,Sleepers,Two gangsters seek revenge on the state jail w...,0,...,0,0,0,0,1,0,0,0,0,0
806,1,1.0,36819.0,False,Family Fantasy ScienceFiction Adventure Comedy,en,/4VZtpwdhHQSa4LUkvujyGAHb1hG.jpg,Time Bandits,Young history buff Kevin can scarcely believe ...,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
ratings_df[ratings_df['userId'] == 1].sort_values(by=['tmdbId'],ascending=True).head(10)

,userId,rating,tmdbId,adult,genres,original_language,poster_path,title,overview,action,...,sentai,state,telescene,the,thriller,tv,view,vision,war,western
581,1,4.0,97.0,False,ScienceFiction Action Adventure,en,/cTF8lpXvAwT1SLH9kVFZZl0AO0D.jpg,Tron,As Kevin Flynn searches for proof that he inve...,1,...,0,0,0,0,0,0,0,0,0,0
442,1,2.5,152.0,False,ScienceFiction Adventure Mystery,en,/ys5Sgpc27mVo3H5zYLPV9LWDq5.jpg,Star Trek: The Motion Picture,When a destructive space entity is spotted app...,0,...,0,0,0,0,0,0,0,0,0,0
259,1,2.0,665.0,False,Action Adventure Drama History,en,/syPMBvvZsADTTRu3UKuxO1Wflq.jpg,Ben-Hur,BenHur is a 1959 epic film directed by William...,1,...,0,0,0,0,0,0,0,0,0,0
305,1,2.0,783.0,False,Drama History,en,/2z9A4FSu1YySrhhcuqkdMIXpgyN.jpg,Gandhi,In the early years of the 20th century Mohanda...,0,...,0,0,0,0,0,0,0,0,0,0
84,1,3.0,819.0,False,Crime Drama Thriller,en,/cDqEv4Fw4JZh2zCfecqw3z09L8z.jpg,Sleepers,Two gangsters seek revenge on the state jail w...,0,...,0,0,0,0,1,0,0,0,0,0
664,1,2.0,847.0,False,Adventure Drama Action,en,/uxB76XmAMEFeeaRqsPAVumn6rTE.jpg,Willow,Fearful of a prophecy stating that a girl chil...,1,...,0,0,0,0,0,0,0,0,0,0
535,1,4.0,1051.0,False,Action Crime Thriller,en,/xwDi4wF0I1L5bSW08Y8GocDGjJP.jpg,The French Connection,Tough narcotics detective Popeye Doyle is in h...,1,...,0,0,0,0,1,0,0,0,0,0
117,1,2.0,1103.0,False,ScienceFiction Action,en,/z1KTbKJh7vSTTXllxxE4DWG63rT.jpg,Escape from New York,In 1997 the island of Manhattan has been walle...,1,...,0,0,0,0,0,0,0,0,0,0
403,1,2.0,1598.0,False,Crime Thriller,en,/4KvrvcqckdupXXO2YnANtyG7QLK.jpg,Cape Fear,Sam Bowden is a smalltown corporate attorney M...,0,...,0,0,0,0,1,0,0,0,0,0
489,1,1.0,3179.0,False,Animation Comedy,en,/2pW1iACRzmb9OOIhzflYYt3S3s9.jpg,Beavis and Butt-Head Do America,Mike Judges slacker duo Beavis and ButtHead wa...,0,...,0,0,0,0,0,0,0,0,0,0


# Clean Na and NaN


In [ ]:
ratings_df.isna().sum() #see columns with na or nan values

userId                 0
rating                 0
tmdbId               213
adult                  0
genres                 0
original_language      0
poster_path            6
title                213
overview              14
action                 0
adventure              0
animation              0
aniplex                0
brosta                 0
carousel               0
cartel                 0
comedy                 0
committee              0
crime                  0
documentary            0
drama                  0
entertainment          0
family                 0
fantasy                0
film                   0
filmworks              0
foreign                0
gohands                0
group                  0
history                0
horror                 0
mardock                0
media                  0
movie                  0
music                  0
mystery                0
odyssey                0
production             0
productions            0
pulser                 0


In [ ]:
ratings_df = ratings_df.dropna()
ratings_df.isna().sum()

userId               0
rating               0
tmdbId               0
adult                0
genres               0
original_language    0
poster_path          0
title                0
overview             0
action               0
adventure            0
animation            0
aniplex              0
brosta               0
carousel             0
cartel               0
comedy               0
committee            0
crime                0
documentary          0
drama                0
entertainment        0
family               0
fantasy              0
film                 0
filmworks            0
foreign              0
gohands              0
group                0
history              0
horror               0
mardock              0
media                0
movie                0
music                0
mystery              0
odyssey              0
production           0
productions          0
pulser               0
rogue                0
romance              0
sciencefiction       0
scramble   

# train-test split

In [ ]:
from sklearn.model_selection import train_test_split

ratings_df, test_df = train_test_split(ratings_df, test_size=0.2)

#ratings_df_csv = ratings_df[['userId', 'rating', 'tmdbId', 'original_language', 'poster_path', 'title', 'overview']]
#ratings_df_csv.to_csv(r'ratings_df_clean.csv', index = False)

# Content Based:
<ol>
  <li>We will use the Word2Vec with the genres of each movie based on the movies each user've seen, so we need a iterable list to tran as says in the <a href="https://radimrehurek.com/gensim/models/word2vec.html">Gensim Word2Vec documentation</a>, so to do that we need:
    <ul>
      <li>A list of the movies and his genres (Because wi will train content based, so we will use the movie id and also the genres) 
      </li>
    </ul>
  </li>
  <li>We will use the <a href ="https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.cosine_similarity.html">sklearn cosine similarity </a> with the description (overview) of each movie, but as we can see in the documentation we need a numei array so we can transform the words using TF-ITF. 
    <ul>
      <li>so we need to preprocess the description of each movie</li>
    </ul>
  </li>
</ol>
Inspired in:
<ol>
<li><a href="https://www.sciencedirect.com/science/article/pii/S1877750318300036">HRS-CE: A hybrid framework to integrate content embeddings in recommender systems for cold start items</a></li>
<li><a href="https://medium.com/@fernando.bugni/using-metadata-with-word2vec-to-get-recommendations-in-movielens-3ae45afaf957">
Using Metadata with Word2Vec to get recommendations in MovieLens</a></li>
<li><a href="https://arxiv.org/pdf/1411.2738.pdf">word2vec Parameter Learning Explained</a></li>
<li><a href="https://medium.com/analytics-vidhya/content-based-recommender-systems-in-python-2b330e01eb80">Content-based Recommender Systems in Python</a></li>
<li><a href="https://www.tandfonline.com/doi/abs/10.4103/02564602.2010.10876780">Content-based File-type Identification Using Cosine Similarity and a Divide-and-Conquer Approach</a></li>
</ol>

In [ ]:
movies_df = ratings_df.copy()
ratings_df = ratings_df[ratings_df['rating'] >= 3] #We will use just the rating of 3 or more
ratings_df.head()

,userId,rating,tmdbId,adult,genres,original_language,poster_path,title,overview,action,...,sentai,state,telescene,the,thriller,tv,view,vision,war,western
56179,607,4.0,4327.0,False,Action Adventure Comedy Crime Thriller,en,/eBzf9d09Vgq2HSVC4fIZm1QNQd.jpg,Charlie's Angels,Aspects of this take on the 1970s hit TV serie...,1,...,0,0,0,0,1,0,0,0,0,0
40872,30,3.0,11236.0,False,Drama Family Fantasy,en,/pI6J5nrFKh4GEUZ1eGP4AEVQOAN.jpg,The Secret Garden,A young British girl born and reared in India ...,0,...,0,0,0,0,0,0,0,0,0,0
64876,371,3.5,8681.0,False,Action Thriller,en,/3zlffXmo7QpVBc17QIJWrRfasVr.jpg,Taken,While vacationing with a friend in Paris an Am...,1,...,0,0,0,0,1,0,0,0,0,0
2714,588,3.0,1642.0,False,Crime Drama Mystery Thriller Action,en,/gKDNaAFzT21cSVeKQop7d1uhoSp.jpg,The Net,Angela Bennett is a freelance software enginee...,1,...,0,0,0,0,1,0,0,0,0,0
8496,418,3.0,268.0,False,Fantasy Action,en,/kBf3g9crrADGMc2AMAMlLBgSm2h.jpg,Batman,The Dark Knight of Gotham City begins his war ...,1,...,0,0,0,0,0,0,0,0,0,0


# Word2Vec (users and genres)

In [ ]:
users = ratings_df['userId'].unique().tolist() #take the users
#print(users)

In [ ]:
movies_by_user = [] #array of dictionary of movies and users
for user in users:
  user_df = ratings_df[ratings_df['userId'] == user] #take the movies whatched by the user
  user_df = user_df[['tmdbId', 'genres']] #take the columns that we need
  #user_df = user_df[['tmdbId', 'overview', 'genres']] #take the columns that we need
  #user_df['genres'] = user_df['overview'] + user_df['genres']
  #user_df = user_df.drop(columns=['overview'])
  user_df = user_df.set_index('tmdbId') 
  movies_by_user.append(user_df.to_dict()) #create the dict movie id : genres
#movies_by_user

In [ ]:
#the genres title is just noise, so we will remove this and use just the conten of each dictionary
train_list = []
for user_dict in movies_by_user:
  train_list.append(user_dict['genres'])
#train_list

In [ ]:
# clean the spaces in the dictionary: ' ' and '    '
for user_dict in train_list:
  values = []
  for key,value in user_dict.items():
    user_dict[key] = value.replace('    ', ' ')

In [ ]:
#conver the dictionary to arrays
final_train_list = []
for user_dict in train_list:
  values = []
  for key,value in user_dict.items():
    for genre in value.strip().split(' '):
      values.append(genre)
      values.append(str(int(key)))
  final_train_list.append(values)
#final_train_list


2. Create and train the model:</li> We will use the <a href="https://radimrehurek.com/gensim/models/word2vec.html"> Word2Vec </a>, so we need:
<ul>
<li> an iterable list</li>
<li> the window</li>
<li> the min_count</li>
<li> the workers</li>
</ul>


In [ ]:
from gensim.test.utils import common_texts
from gensim.models.word2vec import Word2Vec

In [ ]:
#create the model
model = Word2Vec(final_train_list, window=5, min_count=2, workers=4)

In [ ]:
#function that returns similar movies
def most_similar_movie(movieId):
    print("Similar of "+ratings_df[ratings_df['tmdbId'] == int(movieId)].iloc[0]['title'])
    return [(int(x[0]), ratings_df[ratings_df['tmdbId'] == int(x[0])].iloc[0]['title']) for x in model.wv.most_similar(movieId)]

In [ ]:
most_similar_movie('7340')

Similar of Carrie


[(794, 'The Omen'),
 (9871, 'Ginger Snaps'),
 (9095, 'Mary Reilly'),
 (948, 'Halloween'),
 (579, 'Jaws 2'),
 (10242, 'What Ever Happened to Baby Jane?'),
 (15516, 'The Last House on the Left'),
 (9740, 'Hannibal'),
 (10493, 'Dead Calm'),
 (9529, 'Candyman')]

In [ ]:
def most_similar_gener(genres):
  count = 0
  for genre in genres:
    if count == 0:
      vector = model[genre]
      count = count + 1
    else:
      vector = model[genre] + vector
  print("Similar of ",list(genres))
  #print(model.wv.most_similar([vector]))
  resp = []
  for x in model.wv.most_similar([vector]):
    try:
      int(x[0])
      resp.append((int(x[0]), ratings_df[ratings_df['tmdbId'] == int(x[0])].iloc[0]['title']))
    except:
      print(x)
  return resp


In [ ]:
most_similar_gener(['Horror', 'Drama', 'Crime'])

Similar of  ['Horror', 'Drama', 'Crime']


C:\Users\EQUIPO\anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """
C:\Users\EQUIPO\anaconda3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


[(3529, 'The Thin Man'),
 (9469, 'He Got Game'),
 (234, 'The Cabinet of Dr. Caligari'),
 (39437, 'Beloved'),
 (9095, 'Mary Reilly'),
 (9740, 'Hannibal'),
 (6217, 'Cat People'),
 (14337, 'Primer'),
 (9276, 'The Faculty'),
 (814, 'An American Werewolf in London')]

# Cosine_similarity (description):
Now we will use the description too:

In [ ]:
movies = movies_df['tmdbId'].unique().tolist() #take the users
#print(movies)

In [ ]:
metadata_df = metadata_df[metadata_df['tmdbId'].isin(movies)]
metadata_df.head(3)

,adult,genres,tmdbId,original_language,poster_path,title,overview
0,False,Animation Comedy Family,862.0,en,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,Toy Story,Led by Woody Andys toys live happily in his ro...
1,False,Adventure Fantasy Family,8844.0,en,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,Jumanji,When siblings Judy and Peter discover an encha...
2,False,Romance Comedy,15602.0,en,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,Grumpier Old Men,A family wedding reignites the ancient feud be...


In [ ]:
#Clean Metadata
from gensim.parsing.preprocessing import remove_stopwords
metadata_df['overview'] = metadata_df['overview'].str.replace(r'[^\w\s]+', '') #remove punctuation
metadata_df['overview'] = metadata_df['overview'].apply(lambda x: remove_stopwords(x))
metadata_df['genres'] = metadata_df['genres'].str.strip()
#metadata_df

In [ ]:
description_df = metadata_df[['tmdbId', 'overview','title']]
description_df

,tmdbId,overview,title
0,862.0,Led Woody Andys toys live happily room Andys b...,Toy Story
1,8844.0,When siblings Judy Peter discover enchanted bo...,Jumanji
2,15602.0,A family wedding reignites ancient feud nextdo...,Grumpier Old Men
3,31357.0,Cheated mistreated stepped women holding breat...,Waiting to Exhale
4,11862.0,Just George Banks recovered daughters wedding ...,Father of the Bride Part II
...,...,...,...
40058,402672.0,Village lad Sarman drawn big bad Mohenjo Daro ...,Mohenjo Daro
40503,391698.0,The band stormed Europe 1963 1964 conquered Am...,The Beatles: Eight Days a Week - The Touring Y...
44821,10991.0,When Molly Hales sadness fathers disappearance...,Pokémon: Spell of the Unknown
44826,12600.0,All favorite Pokémon characters joined time le...,Pokémon 4Ever: Celebi - Voice of the Forest


TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words='english')#tfidf instance

#Construct the required TF-IDF matrix by applying the fit_transform method on the overview feature
overview_matrix = tfidf.fit_transform(description_df['overview'])
overview_matrix.shape

(8371, 32310)

In [ ]:
from sklearn.metrics.pairwise import linear_kernel
similarity_matrix = linear_kernel(overview_matrix,overview_matrix)
similarity_matrix

array([[1.        , 0.01759393, 0.        , ..., 0.        , 0.0063767 ,
        0.        ],
       [0.01759393, 1.        , 0.02529325, ..., 0.01210229, 0.0185096 ,
        0.        ],
       [0.        , 0.02529325, 1.        , ..., 0.        , 0.00770895,
        0.01341887],
       ...,
       [0.        , 0.01210229, 0.        , ..., 1.        , 0.03724275,
        0.        ],
       [0.0063767 , 0.0185096 , 0.00770895, ..., 0.03724275, 1.        ,
        0.        ],
       [0.        , 0.        , 0.01341887, ..., 0.        , 0.        ,
        1.        ]])

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
#movies index mapping
mapping = pd.Series(description_df.index,index = description_df['tmdbId'])
mapping

tmdbId
862.0           0
8844.0          1
15602.0         2
31357.0         3
11862.0         4
            ...  
402672.0    40058
391698.0    40503
10991.0     44821
12600.0     44826
265189.0    45265
Length: 8371, dtype: int64

In [ ]:
def most_similar_description(movie_input):
    movie_input = int(movie_input)
    movie_index = mapping[movie_input]
    #get similarity values with other movies
    #similarity_score is the list of index and similarity matrix
    similarity_score = list(enumerate(similarity_matrix[movie_index]))
    #sort in descending order the similarity score of movie inputted with all the other movies
    similarity_score = sorted(similarity_score, key=lambda x: x[1], reverse=True)
    # Get the scores of the 15 most similar movies. Ignore the first movie.
    similarity_score = similarity_score[1:15]
    #return movie names using the mapping series
    movie_indices = [i[0] for i in similarity_score]
    movie_ids = description_df['tmdbId'].iloc[movie_indices].tolist()
    return ([(int(movie),description_df[description_df['tmdbId'] == movie]['title'].to_string(index=False).strip()) for movie in movie_ids])

In [ ]:
most_similar_description('15')

[(10728, 'Faust'),
 (26957, 'Puppet Master 4'),
 (26618, 'Clay Pigeons'),
 (49948, 'Fantasia 2000'),
 (91076, 'Illuminata'),
 (66600, 'Sweet Liberty'),
 (44916, "Spencer's Mountain"),
 (15067, 'The Good, The Bad, The Weird'),
 (87078, 'Suture'),
 (22975, 'Richard Pryor: Live in Concert'),
 (10657, 'Needful Things'),
 (315, 'Faster, Pussycat! Kill! Kill!'),
 (124623, 'I Know That Voice'),
 (4588, 'Lust, Caution')]

In [ ]:
def get_genres(movie_id):
    return metadata_df[metadata_df['tmdbId'] == movie_id].genres.to_string(index=False).strip()
print(get_genres(98))

Action Drama Adventure


# Content Based Integration

In [ ]:
def content_based(movie_id):
    #search by simmilar description
    description_sim = most_similar_description(movie_id)
    #search by simmilar movie name
    try:
        movie_sim = most_similar_movie(movie_id)
    except:
        movie_sim = []
    set_1 = set(description_sim)
    set_2 = set(movie_sim)
    moviesSet2_notin_Set1 = list(set_2 - set_1)
    combined_sim = description_sim + moviesSet2_notin_Set1
    #search by simmilar genres
    genres_string = get_genres(int(movie_id)) #take the genres of the movie
    genres = genres_string.split()
    #print(genres)
    #use the function
    try:
        genres_sim = most_similar_gener(genres)
    except:
        genres_sim = []
    set_1 = set(combined_sim)
    set_2 = set(genres_sim)
    moviesSet2_notin_Set1 = list(set_2 - set_1)
    combined_sim = combined_sim + moviesSet2_notin_Set1
    #don't repeat movies
    combined_similar_movies = list(set(combined_sim))
    return combined_similar_movies

In [ ]:
"""
for title in metadata_df['title']:
    if "The Imi" in title:
        print(f"We Have the movie: {title}")
"""
for search_movie in ratings_df['title']:
    if "The Conj" in search_movie:
        print (f'the movie:{search_movie}')


the movie:The Conjuring 2
the movie:The Conjuring
the movie:The Conjuring
the movie:The Conjuring 2


In [ ]:
content_based('98')

Similar of Gladiator
Similar of  ['Action', 'Drama', 'Adventure']
('Action', 0.7894898056983948)
('Adventure', 0.7549953460693359)


C:\Users\EQUIPO\anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """
C:\Users\EQUIPO\anaconda3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


[(2789, 'The Chronicles of Riddick'),
 (9093, 'The Four Feathers'),
 (25095, 'Wuthering Heights'),
 (82687, 'The Guilt Trip'),
 (19214, 'Passion of Mind'),
 (9840, 'The Dreamlife of Angels'),
 (1995, 'Lara Croft: Tomb Raider'),
 (11006, 'Smokey and the Bandit'),
 (15045, 'Fat Albert'),
 (11777, 'I.Q.'),
 (9342, 'The Mask of Zorro'),
 (37724, 'Skyfall'),
 (2661, 'Batman'),
 (8856, 'The Karate Kid, Part II'),
 (646, 'Dr. No'),
 (166424, 'Fantastic Four'),
 (10680, 'D3: The Mighty Ducks'),
 (209263, 'Enough Said'),
 (929, 'Godzilla'),
 (421, 'The Life Aquatic with Steve Zissou'),
 (1829, 'The Story of Adele H'),
 (73873, 'Albert Nobbs'),
 (76338, 'Thor: The Dark World'),
 (254, 'King Kong'),
 (9313, 'The Man in the Iron Mask'),
 (8488, 'Hitch'),
 (27118, 'The Hound of the Baskervilles'),
 (57212, 'War Horse'),
 (85328, 'The Pompatus of Love'),
 (26317, 'Man with a Movie Camera'),
 (45608, 'The World of Suzie Wong')]

# collaborative filtering:
<ol>
    <li>define the train data:</li> We will use the <a href="https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.corrwith.html">pandas correlation</a> , so we need the matrix of users and movies fill by ratings to compare with and compare.
</ol>
Inspired in:
<ol>
<li><a href="https://reader.elsevier.com/reader/sd/pii/S0950705113003560?token=AEB79FC170F261E8FED93BE7B1523DBC14153B14255132400F29C3C3F8DD439F1CA85CD5349847C02FAB168B8320FBF9">A new user similarity model to improve the accuracy of collaborative filtering</a></li>
<li><a href="https://youtu.be/BDJmJnrlaO8">Cosine Similarity (User-User) </a></li>
    
</ol>


# user

In [ ]:
for title in metadata_df['title']:
    if "The Imi" in title:
        print(f"We Have the movie: {title}")

We Have the movie: The Imitation Game


In [ ]:
metadata_df[metadata_df['title']=='The Imitation Game'][['tmdbId','title']]
metadata_df

,adult,genres,tmdbId,original_language,poster_path,title,overview
0,False,Animation Comedy Family,862.0,en,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,Toy Story,Led Woody Andys toys live happily room Andys b...
1,False,Adventure Fantasy Family,8844.0,en,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,Jumanji,When siblings Judy Peter discover enchanted bo...
2,False,Romance Comedy,15602.0,en,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,Grumpier Old Men,A family wedding reignites ancient feud nextdo...
3,False,Comedy Drama Romance,31357.0,en,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,Waiting to Exhale,Cheated mistreated stepped women holding breat...
4,False,Comedy,11862.0,en,/e64sOI48hQXyru7naBFyssKFxVd.jpg,Father of the Bride Part II,Just George Banks recovered daughters wedding ...
...,...,...,...,...,...,...,...
40004,False,Thriller Romance,392572.0,hi,/q1lrN6ZrIsOs077lQB86aPGKZRF.jpg,Rustom,Rustom Pavri honourable officer Indian Navy sh...
40058,False,Adventure Drama History Romance,402672.0,hi,/q2XVemXiWSa18mbaVpI3rbLXG2u.jpg,Mohenjo Daro,Village lad Sarman drawn big bad Mohenjo Daro ...
44821,False,Adventure Fantasy Animation Action Family,10991.0,ja,/5ILjS6XB5deiHop8SXPsYxXWVPE.jpg,Pokémon: Spell of the Unknown,When Molly Hales sadness fathers disappearance...
44826,False,Adventure Fantasy Animation ScienceFiction Family,12600.0,ja,/bqL0PVHbQ8Jmw3Njcl38kW0CoeM.jpg,Pokémon 4Ever: Celebi - Voice of the Forest,All favorite Pokémon characters joined time le...


In [ ]:
movies_df = movies_df[['title','tmdbId', 'rating','userId']]

user = [
    #movie : rating
    {'tmdbId':862, 'rating':3}, #toys story
    {'tmdbId':8844, 'rating':  2}, #jumanji
    {'tmdbId':123025, 'rating': 5}, #batman
    {'tmdbId':142061, 'rating': 4.9}, #batman
    {'tmdbId':205596, 'rating': 5}, #the imitation game
    {'tmdbId':58574, 'rating': 4.5}, #sherlock
    {'tmdbId':10528, 'rating': 4.9},#sherlock
    {'tmdbId':82702, 'rating': 4.8}, #How to train
    {'tmdbId':19995, 'rating': 2}, #avatar
    {'tmdbId':18239, 'rating': 1}, #The Twilight
    {'tmdbId':24021, 'rating': 1.2}, #The Twilight
    {'tmdbId':935, 'rating': 4}, #Dr. Strange
    {'tmdbId':76338, 'rating': 5}, #Thor
    {'tmdbId':12, 'rating': 3.8},#nemo
    {'tmdbId':13, 'rating': 4.2}#forest gump
]
user = pd.DataFrame(user)
user

,tmdbId,rating
0,862,3.0
1,8844,2.0
2,123025,5.0
3,142061,4.9
4,205596,5.0
5,58574,4.5
6,10528,4.9
7,82702,4.8
8,19995,2.0
9,18239,1.0


In [ ]:
final_traing_df = movies_df.copy()
movies_df.head()

,title,tmdbId,rating,userId
5193,The Mask,854.0,5.0,225
12997,The Dark Knight,155.0,5.0,62
4513,Forrest Gump,13.0,3.5,77
65372,The Curious Case of Benjamin Button,4922.0,4.5,371
80860,Jingle All the Way,9279.0,3.0,518


In [ ]:
def add_user(user, movies_df):
    #add titles
    titles = user['tmdbId'].tolist()
    titles = [metadata_df[metadata_df['tmdbId'] == title]['title'].to_string(index=False).strip() for title in titles]
    user['title'] = titles
    #add user id
    user_id = movies_df['userId'].max()+1
    user['userId'] = user_id
    movies_df = movies_df.append(user, ignore_index = True)
    return user, movies_df

In [ ]:
user, movies_df = add_user(user, movies_df)
user

,tmdbId,rating,title,userId
0,862,3.0,Toy Story,672
1,8844,2.0,Jumanji,672
2,123025,5.0,"Batman: The Dark Knight Returns, Part 1",672
3,142061,4.9,"Batman: The Dark Knight Returns, Part 2",672
4,205596,5.0,The Imitation Game,672
5,58574,4.5,Sherlock Holmes: A Game of Shadows,672
6,10528,4.9,Sherlock Holmes,672
7,82702,4.8,How to Train Your Dragon 2,672
8,19995,2.0,Avatar,672
9,18239,1.0,The Twilight Saga: New Moon,672


In [ ]:
movies_df.sort_values(by=['userId'],ascending=False).head(10)

,title,tmdbId,rating,userId
79878,Forrest Gump,13.0,4.2,672
79873,The Twilight Saga: New Moon,18239.0,1.0,672
79867,"Batman: The Dark Knight Returns, Part 2",142061.0,4.9,672
79865,Jumanji,8844.0,2.0,672
79864,Toy Story,862.0,3.0,672
79868,The Imitation Game,205596.0,5.0,672
79869,Sherlock Holmes: A Game of Shadows,58574.0,4.5,672
79870,Sherlock Holmes,10528.0,4.9,672
79871,How to Train Your Dragon 2,82702.0,4.8,672
79872,Avatar,19995.0,2.0,672


In [ ]:
users_with_movies_common = {}
users = movies_df['userId'].unique().tolist() #take the users

for user_ in users:
  user_df = movies_df[movies_df['userId'] == user_] #take the movies whatched by the user
  user_df = user_df[['tmdbId', 'userId']] #take the columns that we need
  user_movies_common = user_df[user_df['tmdbId'].isin(user['tmdbId'].tolist())]
  users_with_movies_common[user_df['userId'].iloc[0]] = len(user_movies_common)  #create the dict movie id : genres

users_with2=[]
for user_id,number_movies in users_with_movies_common.items():
    if number_movies > 2:
        users_with2.append(user_id)
users_with2

[77,
 371,
 23,
 457,
 73,
 268,
 353,
 452,
 30,
 468,
 380,
 15,
 664,
 78,
 580,
 213,
 624,
 311,
 585,
 480,
 525,
 200,
 564,
 615,
 94,
 298,
 26,
 125,
 240,
 575,
 212,
 426,
 654,
 251,
 313,
 481,
 232,
 72,
 547,
 262,
 428,
 97,
 379,
 48,
 128,
 577,
 522,
 312,
 56,
 328,
 563,
 505,
 402,
 119,
 433,
 503,
 68,
 562,
 471,
 134,
 608,
 287,
 185,
 205,
 220,
 553,
 187,
 201,
 529,
 88,
 99,
 510,
 250,
 620,
 418,
 537,
 20,
 479,
 345,
 253,
 199,
 146,
 149,
 430,
 584,
 382,
 126,
 130,
 92,
 19,
 13,
 574,
 475,
 177,
 542,
 178,
 355,
 157,
 417,
 531,
 339,
 316,
 500,
 138,
 641,
 283,
 69,
 273,
 486,
 333,
 520,
 238,
 168,
 324,
 625,
 501,
 104,
 84,
 660,
 456,
 600,
 672]

In [ ]:
users_with2 = pd.DataFrame(users_with2, columns=['userId'])
users_with2

,userId
0,77
1,371
2,23
3,457
4,73
...,...
117,84
118,660
119,456
120,600


In [ ]:
#users that saw 3 common movies or more
movies_df = movies_df[movies_df['userId'].isin(users_with2['userId'].tolist())]
movies_df

,title,tmdbId,rating,userId
2,Forrest Gump,13.0,3.5,77
3,The Curious Case of Benjamin Button,4922.0,4.5,371
6,Ran,11645.0,4.5,23
7,La Haine,406.0,3.5,457
9,I Am Sam,10950.0,4.5,73
...,...,...,...,...
79874,The Twilight Saga: Eclipse,24021.0,1.2,672
79875,Dr. Strangelove or: How I Learned to Stop Worr...,935.0,4.0,672
79876,Thor: The Dark World,76338.0,5.0,672
79877,Finding Nemo,12.0,3.8,672


In [ ]:
#creating the user-item interaction matrix
movie_matrix_UII = movies_df.pivot_table(index='tmdbId', columns='userId', values='rating')
movie_matrix_UII.head(5)

userId,13,15,19,20,23,26,30,48,56,68,...,608,615,620,624,625,641,654,660,664,672
tmdbId,,,,,,,,,,,,,,,,,,,,,
5.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6.0,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11.0,NaN,5.0,NaN,1.5,NaN,3.0,4.0,NaN,2.0,3.0,...,NaN,3.5,4.0,5.0,NaN,NaN,5.0,4.5,4.5,NaN
12.0,4.5,1.0,NaN,NaN,NaN,NaN,NaN,NaN,4.0,3.0,...,NaN,4.5,1.0,3.5,NaN,NaN,5.0,NaN,NaN,3.8
13.0,5.0,NaN,NaN,2.0,4.5,4.5,5.0,NaN,4.0,NaN,...,3.0,4.0,NaN,3.0,NaN,5.0,4.0,NaN,NaN,4.2


In [ ]:
#take my user
myUser = movies_df['userId'].max()
#Take the column of the user we want
myUser = movie_matrix_UII[myUser]
myUser

tmdbId
5.0         NaN
6.0         NaN
11.0        NaN
12.0        3.8
13.0        4.2
           ... 
384798.0    NaN
390989.0    NaN
399106.0    NaN
401387.0    NaN
410921.0    NaN
Name: 672, Length: 7060, dtype: float64

In [ ]:
#calculate the correlation
similar_to_user = movie_matrix_UII.corrwith(myUser)

In [ ]:
#create the data frame
corr_user = pd.DataFrame(similar_to_user, columns=['Correlation'])
#drop the na
corr_user.dropna(inplace=True)
#drop the user:
myUserId = movies_df['userId'].max()
corr_user = corr_user.drop([myUserId]) 
#see the correlation df
corr_user.head(50)

,Correlation
userId,
13,-0.188982
15,-0.390126
19,0.866025
20,-0.423415
23,0.987829
26,-0.177580
30,0.713506
48,-0.542660
56,0.000000


In [ ]:
#Most rated movies
corr_user['Correlation'] = corr_user['Correlation']

corr_user = corr_user.sort_values('Correlation', ascending=False)
corr_user.head(50)

,Correlation
userId,
563,0.999841
418,0.997949
23,0.987829
220,0.987829
88,0.985329
283,0.981981
379,0.981981
313,0.970725
125,0.950586


In [ ]:
corr_user = corr_user[0:10]
corr_user

,Correlation
userId,
563,0.999841
418,0.997949
23,0.987829
220,0.987829
88,0.985329
283,0.981981
379,0.981981
313,0.970725
125,0.950586


In [ ]:
movies_df = movies_df[movies_df['userId'].isin(corr_user.index.values.tolist())]
movies_df['Correlation'] = movies_df['userId'].apply(lambda x: corr_user['Correlation'].loc[x])
movies_df

C:\Users\EQUIPO\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,title,tmdbId,rating,userId,Correlation
6,Ran,11645.0,4.5,23,0.987829
73,Spider-Man,557.0,4.0,125,0.950586
101,Gone in Sixty Seconds,9679.0,3.0,125,0.950586
114,Memento,77.0,3.5,313,0.970725
154,The Silence of the Lambs,274.0,4.0,379,0.981981
...,...,...,...,...,...
79639,The Incredibles,9806.0,4.5,313,0.970725
79658,Throne of Blood,3777.0,4.0,23,0.987829
79729,Inception,27205.0,4.0,563,0.999841
79854,Platoon,792.0,4.0,313,0.970725


In [ ]:
movies_df['weight'] = (movies_df['rating']*movies_df['Correlation'])/5
movies_df = movies_df.sort_values('weight', ascending=False)
movies_df

C:\Users\EQUIPO\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,title,tmdbId,rating,userId,Correlation,weight
36029,Pulp Fiction,680.0,5.0,563,0.999841,0.999841
59759,Full Metal Jacket,600.0,5.0,563,0.999841,0.999841
29396,The Disappearance of Haruhi Suzumiya,36865.0,5.0,563,0.999841,0.999841
20852,Interstellar,157336.0,5.0,563,0.999841,0.999841
65505,Shrek,808.0,5.0,563,0.999841,0.999841
...,...,...,...,...,...,...
61775,There's Something About Mary,544.0,0.5,417,0.944911,0.094491
59209,Aladdin,812.0,0.5,417,0.944911,0.094491
45412,The Little Mermaid,10144.0,0.5,417,0.944911,0.094491
24850,Austin Powers: The Spy Who Shagged Me,817.0,0.5,417,0.944911,0.094491


In [ ]:
colaborative_filter_df=movies_df[['tmdbId','title']][0:10]
colaborative_filter_df.head(15)

,tmdbId,title
36029,680.0,Pulp Fiction
59759,600.0,Full Metal Jacket
29396,36865.0,The Disappearance of Haruhi Suzumiya
20852,157336.0,Interstellar
65505,808.0,Shrek
64580,145247.0,The 100 Year-Old Man Who Climbed Out the Windo...
22700,205596.0,The Imitation Game
38846,550.0,Fight Club
56237,18491.0,The End of Evangelion
56124,62.0,2001: A Space Odyssey


In [ ]:
#give the format from dataframe to list of tuples[(movieid,movie),(...)]
colaborative_movies = colaborative_filter_df['tmdbId'].unique().tolist()
colaborative_filter_movies = []
for movie in colaborative_movies:
    movie = int(movie)
    colaborative_filter_movies.append((movie,colaborative_filter_df[colaborative_filter_df['tmdbId'] == movie]['title'].to_string(index=False)))
colaborative_filter_movies

[(680, ' Pulp Fiction'),
 (600, ' Full Metal Jacket'),
 (36865, ' The Disappearance of Haruhi Suzumiya'),
 (157336, ' Interstellar'),
 (808, ' Shrek'),
 (145247, ' The 100 Year-Old Man Who Climbed Out the Windo...'),
 (205596, ' The Imitation Game'),
 (550, ' Fight Club'),
 (18491, ' The End of Evangelion'),
 (62, ' 2001: A Space Odyssey')]

In [ ]:
def get_collaborative_filtering(user):
    #to keep the original df
    movies_df = final_traing_df.copy()
    
    if isinstance(user, int):
        user = movies_df[movies_df['userId'] == user]
        
    else:
        #take the dict user, create a dataframe os user and add the dataframe to movies_df
        user, movies_df = add_user(user, movies_df)

    users_with_movies_common = {}
    users = movies_df['userId'].unique().tolist() #take the users

    for user_ in users:
      user_df = movies_df[movies_df['userId'] == user_] #take the movies whatched by the user
      user_df = user_df[['tmdbId', 'userId']] #take the columns that we need
      user_movies_common = user_df[user_df['tmdbId'].isin(user['tmdbId'].tolist())]
      users_with_movies_common[user_df['userId'].iloc[0]] = len(user_movies_common)  #create the dict movie id : genres

    users_with2=[]
    for user_id,number_movies in users_with_movies_common.items():
        if number_movies > 2:
            users_with2.append(user_id)
    users_with2 = pd.DataFrame(users_with2, columns=['userId'])
    #users that saw 3 common movies or more
    movies_df = movies_df[movies_df['userId'].isin(users_with2['userId'].tolist())]
    #creating the user-item interaction matrix
    movie_matrix_UII = movies_df.pivot_table(index='tmdbId', columns='userId', values='rating')
    #take my user
    myUser = movies_df['userId'].max()
    #Take the column of the user we want
    myUser = movie_matrix_UII[myUser]
    #calculate the correlation
    similar_to_user = movie_matrix_UII.corrwith(myUser)
    #create the data frame
    corr_user = pd.DataFrame(similar_to_user, columns=['Correlation'])
    #drop the na
    corr_user.dropna(inplace=True)
    #drop the user:
    myUserId = movies_df['userId'].max()
    corr_user = corr_user.drop([myUserId]) 
    #Most rated movies
    corr_user['Correlation'] = corr_user['Correlation']
    corr_user = corr_user.sort_values('Correlation', ascending=False)
    #take just the most correlated
    corr_user = corr_user[0:10]
    #take from movies_df just the movies with highest correlation
    movies_df = movies_df[movies_df['userId'].isin(corr_user.index.values.tolist())]
    #add the correlation column to the dataframe
    movies_df['Correlation'] = movies_df['userId'].apply(lambda x: corr_user['Correlation'].loc[x])
    #calculate the weight with the rating
    movies_df['weight'] = (movies_df['rating']*movies_df['Correlation'])/5
    #sort by weight
    movies_df = movies_df.sort_values('weight', ascending=False)
    #take the top movies:
    colaborative_filter_df=movies_df[['tmdbId','title']][0:10]
    #give the format to the movies
    colaborative_movies = colaborative_filter_df['tmdbId'].unique().tolist()
    colaborative_filter_movies = []
    for movie in colaborative_movies:
        movie = int(movie)
        colaborative_filter_movies.append((movie,colaborative_filter_df[colaborative_filter_df['tmdbId'] == movie]['title'].to_string(index=False)))
    return colaborative_filter_movies
    

In [ ]:

user2 = [
    #movie : rating
    {'tmdbId':862, 'rating':3}, #toys story
    {'tmdbId':8844, 'rating':  2}, #jumanji
    {'tmdbId':123025, 'rating': 5}, #batman
    {'tmdbId':142061, 'rating': 4.9}, #batman
    {'tmdbId':205596, 'rating': 5}, #the imitation game
    {'tmdbId':58574, 'rating': 4.5}, #sherlock
    {'tmdbId':10528, 'rating': 4.9},#sherlock
    {'tmdbId':82702, 'rating': 4.8}, #How to train
    {'tmdbId':19995, 'rating': 2}, #avatar
    {'tmdbId':18239, 'rating': 1}, #The Twilight
    {'tmdbId':24021, 'rating': 1.2}, #The Twilight
    {'tmdbId':935, 'rating': 4}, #Dr. Strange
    {'tmdbId':76338, 'rating': 5}, #Thor
    {'tmdbId':12, 'rating': 3.8},#nemo
    {'tmdbId':13, 'rating': 4.2}#forest gump
]
user2 = pd.DataFrame(user)

prediction = get_collaborative_filtering(15)
prediction

C:\Users\EQUIPO\anaconda3\lib\site-packages\numpy\lib\function_base.py:2526: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar)
C:\Users\EQUIPO\anaconda3\lib\site-packages\numpy\lib\function_base.py:2455: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)


[(11113, ' My Fair Lady'),
 (24795, ' Angels in the Outfield'),
 (221, ' Rebel Without a Cause'),
 (2502, ' The Bourne Supremacy'),
 (380, ' Rain Man'),
 (238, ' The Godfather'),
 (240, ' The Godfather: Part II'),
 (1124, ' The Prestige'),
 (1574, ' Chicago'),
 (568, ' Apollo 13')]

# Hybrid Recommendation System

In [ ]:
def get_hybrid_recommendation(user,movie):
    movie = str(movie)
    content_based_movies = content_based(movie)
    
    #to keep the original df
    movies_df = final_traing_df.copy()
    #take user:
    if isinstance(user, int):
        user_inst = movies_df[movies_df['userId'] == user]
        if len(user_inst.index) > 6:
            collaborative_filter_movies = get_collaborative_filtering(user)
            #add the content_based and collaborative movies
            set_1 = set(content_based_movies)
            set_2 = set(collaborative_filter_movies)
            moviesSet2_notin_Set1 = list(set_2 - set_1)
            combined_sim = content_based_movies + moviesSet2_notin_Set1
            #don't repeat movies
            combined_similar_movies = list(set(combined_sim))
            #if the list has more then 10 movies just the first 10
            """
            if len(combined_similar_movies)>21:
                return combined_similar_movies[0:20]
            """
            return combined_similar_movies

    if len(user.index) > 6:
        collaborative_filter_movies = get_collaborative_filtering(user)
        #add the content_based and collaborative movies
        set_1 = set(content_based_movies)
        set_2 = set(collaborative_filter_movies)
        moviesSet2_notin_Set1 = list(set_2 - set_1)
        combined_sim = content_based_movies + moviesSet2_notin_Set1
        #don't repeat movies
        combined_similar_movies = list(set(combined_sim))
        #if the list has more then 10 movies just the first 10
        """
        if len(combined_similar_movies)>21:
            return combined_similar_movies[0:20]
        return combined_similar_movies
        """
    #if the list has more then 10 movies just the first 10
    """
    if len(content_based_movies)>21:
        return content_based_movies[0:20]
    """
    #if not enougth movies, just return the content based movies
    return content_based_movies
        

In [ ]:
get_hybrid_recommendation(user,'12110')

Similar of Dracula: Dead and Loving It
Similar of  ['Comedy', 'Horror']
('Horror', 0.8201881051063538)


C:\Users\EQUIPO\anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """
C:\Users\EQUIPO\anaconda3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


[(927, 'Gremlins'),
 (10577, 'Dracula 2000'),
 (11206, 'Wait Until Dark'),
 (16288, 'Creepshow 2'),
 (794, 'The Omen'),
 (2654, 'In the Mouth of Madness'),
 (25239, 'I Spit on Your Grave'),
 (6217, 'Cat People'),
 (377, 'A Nightmare on Elm Street'),
 (32076, 'The Believers'),
 (948, 'Halloween'),
 (6114, 'Dracula'),
 (4488, 'Friday the 13th'),
 (763, 'Braindead'),
 (9095, 'Mary Reilly'),
 (25944, 'Blood for Dracula'),
 (11452, 'National Lampoon’s Van Wilder'),
 (1547, 'The Lost Boys'),
 (36586, 'Blade II'),
 (9426, 'The Fly'),
 (10925, 'The Return of the Living Dead'),
 (3053, 'The Fearless Vampire Killers'),
 (21588, 'Cemetery Man'),
 (11868, 'Horror of Dracula'),
 (34574, 'Nadja'),
 (6404, 'Nosferatu the Vampyre'),
 (19901, 'Daybreakers'),
 (11336, 'The Dead Zone'),
 (7131, 'Van Helsing'),
 (13815, 'Fresh'),
 (9359, 'Maverick')]

In [ ]:
get_hybrid_recommendation(15,'12110')

Similar of Dracula: Dead and Loving It
Similar of  ['Comedy', 'Horror']
('Horror', 0.8201881051063538)


C:\Users\EQUIPO\anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """
C:\Users\EQUIPO\anaconda3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Users\EQUIPO\anaconda3\lib\site-packages\numpy\lib\function_base.py:2526: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar)
C:\Users\EQUIPO\anaconda3\lib\site-packages\numpy\lib\function_base.py:2455: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)


[(927, 'Gremlins'),
 (10577, 'Dracula 2000'),
 (16288, 'Creepshow 2'),
 (11206, 'Wait Until Dark'),
 (794, 'The Omen'),
 (380, ' Rain Man'),
 (2654, 'In the Mouth of Madness'),
 (25239, 'I Spit on Your Grave'),
 (6217, 'Cat People'),
 (377, 'A Nightmare on Elm Street'),
 (24795, ' Angels in the Outfield'),
 (32076, 'The Believers'),
 (11113, ' My Fair Lady'),
 (240, ' The Godfather: Part II'),
 (6114, 'Dracula'),
 (4488, 'Friday the 13th'),
 (763, 'Braindead'),
 (9095, 'Mary Reilly'),
 (25944, 'Blood for Dracula'),
 (11452, 'National Lampoon’s Van Wilder'),
 (2502, ' The Bourne Supremacy'),
 (221, ' Rebel Without a Cause'),
 (1547, 'The Lost Boys'),
 (568, ' Apollo 13'),
 (1574, ' Chicago'),
 (36586, 'Blade II'),
 (9426, 'The Fly'),
 (10925, 'The Return of the Living Dead'),
 (3053, 'The Fearless Vampire Killers'),
 (7131, 'Van Helsing'),
 (21588, 'Cemetery Man'),
 (11868, 'Horror of Dracula'),
 (34574, 'Nadja'),
 (6404, 'Nosferatu the Vampyre'),
 (19901, 'Daybreakers'),
 (11336, 'The 

In [ ]:
from sklearn.metrics import mean_squared_error
test_df['likes'] = test_df['rating'].apply(lambda x: 1 if (x >= 3) else 0)
test_df

,userId,rating,tmdbId,adult,genres,original_language,poster_path,title,overview,action,...,state,telescene,the,thriller,tv,view,vision,war,western,likes
20088,15,3.0,10890.0,False,Action Comedy,en,/nqfLX1bJLUlZnfflqpUIWrGQwSv.jpg,Stripes,John Winger an indolent sad sack in his 30s im...,1,...,0,0,0,0,0,0,0,0,0,1
1740,389,4.0,2054.0,False,Music Drama Family,en,/jYEKdpVvRV3KdyHMUVYyg0KTVSY.jpg,Mr. Holland's Opus,In 1965 passionate musician Glenn Holland take...,0,...,0,0,0,0,0,0,0,0,0,1
34782,528,3.5,73.0,False,Drama,en,/fXepRAYOx1qC3wju7XdDGx60775.jpg,American History X,Derek Vineyard is paroled after serving 3 year...,0,...,0,0,0,0,0,0,0,0,0,1
24319,358,1.0,15602.0,False,Romance Comedy,en,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,Grumpier Old Men,A family wedding reignites the ancient feud be...,0,...,0,0,0,0,0,0,0,0,0,0
14600,577,5.0,78.0,False,ScienceFiction Drama Thriller,en,/p64TtbZGCElxQHpAMWmDHkWJlH2.jpg,Blade Runner,In the smogchoked dystopian Los Angeles of 201...,0,...,0,0,0,1,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21301,612,4.0,9325.0,False,Family Animation Adventure,en,/1AreNcL9s24Qfghbzomy0IlzCFk.jpg,The Jungle Book,The boy Mowgli makes his way to the manvillage...,0,...,0,0,0,0,0,0,0,0,0,1
94676,255,3.5,34723.0,False,Action Adventure Drama Family,en,/o2Q6bPzAkWjScB1H3kuFpfjyhS.jpg,The Adventures of Huck Finn,Climb aboard for an extraordinary version of M...,1,...,0,0,0,0,0,0,0,0,0,1
11298,150,2.0,620.0,False,Comedy Fantasy,en,/3FS3oBdorgczgfCkFi2u8ZTFfpS.jpg,Ghostbusters,After losing their academic posts at a prestig...,0,...,0,0,0,0,0,0,0,0,0,0
79029,624,4.0,5924.0,False,Crime Drama,en,/q1tmTQvEWbJgf8eccX1XWXxWKkK.jpg,Papillon,A man befriends a fellow criminal as the two o...,0,...,0,0,0,0,0,0,0,0,0,1


In [ ]:
import ml_metrics as metrics
movies_train_users = final_traing_df.userId.unique().tolist()
movies_train_list = final_traing_df.tmdbId.unique().tolist()

#print(movies_train_users)
test_df.shape

(19966, 55)

In [ ]:
test_df = test_df[test_df['userId'].isin(movies_train_list)]
test_df.shape

(14054, 55)

In [ ]:
true_positives = []
false_positives = []
not_seen_movies = []

for user in movies_train_users:
    user_test_df = test_df[test_df['userId'] == user]
    final_traing_user = final_traing_df[final_traing_df['userId'] == user ]
    try:
        top_rated_movie = final_traing_user.sort_values(by=['rating'],ascending=False).tmdbId.iloc[0]
        top_rated_movie = str(int(top_rated_movie))
        recommendations = get_hybrid_recommendation(user,top_rated_movie)
        recommendations = [recommendation[0] for recommendation in recommendations]
        user_test_df = user_test_df[['tmdbId','likes']]
        #print(user_tet_df)
        
        #precision:
        #print(f'the length of recommendations is: {len(recommendations)}')
        user_test_df_inrecommendations = user_test_df[user_test_df['tmdbId'].isin(recommendations)]
        user_test_movies = user_test_df_inrecommendations['likes'].tolist()
        #print(f'user test movies: {user_test_movies}')
        try:
            true_positives_count = 0
            false_positives_count = 0
            for movie in user_test_movies:
                print(movie)
                if movie == 1:
                    true_positives_count = true_positives_count + 1
                if movie == 0:
                    false_positives_count = false_positives_count + 1
            true_positives.append(true_positives_count)
            false_positives.append(false_positives_count)
            #not seen movies            
            not_seen_movies.append(len(recommendations)-len(user_test_movies)) 
        except:
            pass
    except:
        pass
    

Similar of The Mask
Similar of  ['Romance', 'Comedy', 'Crime', 'Fantasy']


C:\Users\EQUIPO\anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """
C:\Users\EQUIPO\anaconda3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Users\EQUIPO\anaconda3\lib\site-packages\numpy\lib\function_base.py:2526: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar)
C:\Users\EQUIPO\anaconda3\lib\site-packages\numpy\lib\function_base.py:2455: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)


Similar of The Terminator
Similar of  ['Action', 'Thriller', 'ScienceFiction']
('ScienceFiction', 0.7537060976028442)
('Action', 0.7274084687232971)


C:\Users\EQUIPO\anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """
C:\Users\EQUIPO\anaconda3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Users\EQUIPO\anaconda3\lib\site-packages\numpy\lib\function_base.py:2526: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar)
C:\Users\EQUIPO\anaconda3\lib\site-packages\numpy\lib\function_base.py:2455: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)


0
1
Similar of Notting Hill
Similar of  ['Romance', 'Comedy', 'Drama']
('Romance', 0.8292446136474609)
('Comedy', 0.6751521229743958)


...

C:\Users\EQUIPO\anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """
C:\Users\EQUIPO\anaconda3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Users\EQUIPO\anaconda3\lib\site-packages\numpy\lib\function_base.py:2526: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar)
C:\Users\EQUIPO\anaconda3\lib\site-packages\numpy\lib\function_base.py:2455: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)
C:\Users\EQUIPO\anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """


Similar of American Beauty
Similar of  ['Drama']
('Drama', 0.9999999403953552)
Similar of Divine Secrets of the Ya-Ya Sisterhood
Similar of  ['Comedy', 'Drama', 'Romance']
('Romance', 0.8292446136474609)
('Comedy', 0.675152063369751)


C:\Users\EQUIPO\anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """
C:\Users\EQUIPO\anaconda3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


Similar of Aladdin
Similar of  ['Animation', 'Family', 'Comedy', 'Adventure', 'Fantasy', 'Romance']
('Family', 0.8321959972381592)


C:\Users\EQUIPO\anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """
C:\Users\EQUIPO\anaconda3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


# precision

In [ ]:
true_positives = sum(true_positives)
print(f'the true positives are: {true_positives}')
false_positives = sum(false_positives)
print(f'the false positives are: {false_positives}')

presicion = true_positives / (true_positives + false_positives)
print(f'the precision is: {presicion}')

the true positives are: 218
the false positives are: 18
the precision is: 0.923728813559322


# Unexpected

In [ ]:
seen_movies = true_positives + false_positives
print(f'the seen movies are: {seen_movies}')
not_seen_movies = sum(not_seen_movies)
print(f'the true positives are: {not_seen_movies}')

unexpected = not_seen_movies/(seen_movies + not_seen_movies)
print(f'the serendipity is: {unexpected}')

the seen movies are: 236
the true positives are: 24156
the serendipity is: 0.9903246966218432
